### Import

In [19]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import  OrdinalEncoder
from sklearn.ensemble import ExtraTreesClassifier

### Data Load

In [20]:
train = pd.read_csv('./train.csv').drop(columns=['ID'])
test = pd.read_csv('./test.csv').drop(columns=['ID'])

In [21]:
X = train.drop('임신 성공 여부', axis=1)
y = train['임신 성공 여부']

### Data Pre-processing

In [22]:
categorical_columns = [
    "시술 시기 코드",
    "시술 당시 나이",
    "시술 유형",
    "특정 시술 유형",
    "배란 자극 여부",
    "배란 유도 유형",
    "단일 배아 이식 여부",
    "착상 전 유전 검사 사용 여부",
    "착상 전 유전 진단 사용 여부",
    "남성 주 불임 원인",
    "남성 부 불임 원인",
    "여성 주 불임 원인",
    "여성 부 불임 원인",
    "부부 주 불임 원인",
    "부부 부 불임 원인",
    "불명확 불임 원인",
    "불임 원인 - 난관 질환",
    "불임 원인 - 남성 요인",
    "불임 원인 - 배란 장애",
    "불임 원인 - 여성 요인",
    "불임 원인 - 자궁경부 문제",
    "불임 원인 - 자궁내막증",
    "불임 원인 - 정자 농도",
    "불임 원인 - 정자 면역학적 요인",
    "불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태",
    "배아 생성 주요 이유",
    "총 시술 횟수",
    "클리닉 내 총 시술 횟수",
    "IVF 시술 횟수",
    "DI 시술 횟수",
    "총 임신 횟수",
    "IVF 임신 횟수",
    "DI 임신 횟수",
    "총 출산 횟수",
    "IVF 출산 횟수",
    "DI 출산 횟수",
    "난자 출처",
    "정자 출처",
    "난자 기증자 나이",
    "정자 기증자 나이",
    "동결 배아 사용 여부",
    "신선 배아 사용 여부",
    "기증 배아 사용 여부",
    "대리모 여부",
    "PGD 시술 여부",
    "PGS 시술 여부"
]

In [23]:
# 카테고리형 컬럼들을 문자열로 변환
for col in categorical_columns:
    X[col] = X[col].astype(str)
    test[col] = test[col].astype(str)

In [24]:
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

X_train_encoded = X.copy()
X_train_encoded[categorical_columns] = ordinal_encoder.fit_transform(X[categorical_columns])

X_test_encoded = test.copy()
X_test_encoded[categorical_columns] = ordinal_encoder.transform(test[categorical_columns])

In [25]:
numeric_columns = [
    "임신 시도 또는 마지막 임신 경과 연수",
    "총 생성 배아 수",
    "미세주입된 난자 수",
    "미세주입에서 생성된 배아 수",
    "이식된 배아 수",
    "미세주입 배아 이식 수",
    "저장된 배아 수",
    "미세주입 후 저장된 배아 수",
    "해동된 배아 수",
    "해동 난자 수",
    "수집된 신선 난자 수",
    "저장된 신선 난자 수",
    "혼합된 난자 수",
    "파트너 정자와 혼합된 난자 수",
    "기증자 정자와 혼합된 난자 수",
    "난자 채취 경과일",
    "난자 해동 경과일",
    "난자 혼합 경과일",
    "배아 이식 경과일",
    "배아 해동 경과일"
]

In [26]:
from sklearn.impute import SimpleImputer

# 결측치 처리 (평균값으로 대체)
num_imputer = SimpleImputer(strategy="mean")

X_train_encoded[numeric_columns] = num_imputer.fit_transform(X_train_encoded[numeric_columns])
X_test_encoded[numeric_columns] = num_imputer.transform(X_test_encoded[numeric_columns])

### 검증

In [27]:
import numpy as np
import pandas as pd
from sklearn.ensemble import ExtraTreesClassifier 
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score 
from tqdm import tqdm 

# 모델 정의
model = ExtraTreesClassifier(random_state=42)

# 교차 검증 설정 (5-Fold)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 진행도 표시를 위한 tqdm 적용
cv_scores = []
for fold, (train_idx, val_idx) in enumerate(tqdm(cv.split(X_train_encoded, y), desc="Cross Validation Progress", total=5)):
    X_train_fold, X_val_fold = X_train_encoded.iloc[train_idx], X_train_encoded.iloc[val_idx]
    y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]
    
    model.fit(X_train_fold, y_train_fold)  # 모델 학습
    y_val_pred = model.predict_proba(X_val_fold)[:, 1]  # ROC-AUC 계산을 위해 확률값 추출
    
    auc_score = roc_auc_score(y_val_fold, y_val_pred)
    cv_scores.append(auc_score)
    
    print(f"Fold {fold+1}: ROC-AUC = {auc_score:.4f}")

# 최종 결과 출력
print(f"\n✅ 5-Fold ROC-AUC 점수 평균: {np.mean(cv_scores):.4f}")
print(f"각 Fold 점수: {cv_scores}")

Cross Validation Progress:  20%|██        | 1/5 [00:18<01:12, 18.20s/it]

Fold 1: ROC-AUC = 0.6908


Cross Validation Progress:  40%|████      | 2/5 [00:36<00:53, 18.00s/it]

Fold 2: ROC-AUC = 0.6920


Cross Validation Progress:  60%|██████    | 3/5 [00:53<00:35, 17.95s/it]

Fold 3: ROC-AUC = 0.6910


Cross Validation Progress:  80%|████████  | 4/5 [01:12<00:18, 18.08s/it]

Fold 4: ROC-AUC = 0.6838


Cross Validation Progress: 100%|██████████| 5/5 [01:30<00:00, 18.11s/it]

Fold 5: ROC-AUC = 0.6906

✅ 5-Fold ROC-AUC 점수 평균: 0.6896
각 Fold 점수: [np.float64(0.690758222769396), np.float64(0.6920458133755459), np.float64(0.690980918208377), np.float64(0.683781069484505), np.float64(0.6905510254670723)]


### Train

In [28]:
model = ExtraTreesClassifier(random_state=42)

model.fit(X_train_encoded, y)

ExtraTreesClassifier(random_state=42)

### Predict

In [29]:
pred_proba = model.predict_proba(X_test_encoded)[:, 1]

### Submission

In [30]:
sample_submission = pd.read_csv('./sample_submission.csv')
sample_submission['probability'] = pred_proba

In [31]:
sample_submission.to_csv('./baseline_submit.csv', index=False)